In [1]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "pastis"
FINAL_DATA_NAME = 'approx1e5-pastis-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "pastis-RGB"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','pastis')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [3]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['test519.npz', 'test525.npz', 'test243.npz', 'test257.npz', 'test531.npz']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [6]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


full-pastis-RGB



# Fourier

In [7]:
#Values obtained from plots in pastisFourierEDA.ipynb
STARTING_VALUE = 0.059498227389561786
ENDING_VALUE = 0.6743233008450027
MULT_FACTOR = 1.08
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(STARTING_VALUE,ENDING_VALUE, MULT_FACTOR)

In [8]:
TRANSFORM = "fourier"
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()


100%|██████████| 1590/1590 [00:05<00:00, 290.28it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:00<00:00, 36.30it/s]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.05687585851000405)",27,"[-32.279766, 46.440998, -18.512537, -116.88595..."
1,2,red,"(0.059498227389561786, 0.06298638865858242)",4,"[-0.63529116, 1.3752662, -0.87903935, -4.08029..."
2,3,red,"(0.06442352540027595, 0.06720566614877052)",4,"[-1.6063294, 2.9262419, 2.7054613, -0.19476253..."
3,4,red,"(0.06987712429686843, 0.07411588266019639)",6,"[0.0577018, -3.0416946, -1.2847528, 0.05495781..."
4,5,red,"(0.07694420157653206, 0.08043461047646094)",6,"[-0.83860844, -4.5440745, 1.5695789, 2.7592669..."


In [9]:
del channel_fr, group_data_map, group_data_map_size

In [10]:
TRANSFORM = "fourier"
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1590/1590 [00:03<00:00, 439.77it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:00<00:00, 37.08it/s] 


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.05687585851000405)",27,"[0.20727114, -33.177452, -23.717556, -104.1030..."
1,2,blue,"(0.059498227389561786, 0.06298638865858242)",4,"[-0.87099785, 0.06491059, -0.69205207, -7.4692..."
2,3,blue,"(0.06442352540027595, 0.06720566614877052)",4,"[-1.7783145, 3.4797366, 4.6696415, -1.2334685,..."
3,4,blue,"(0.06987712429686843, 0.07411588266019639)",6,"[0.18932088, -3.8716536, -2.9208791, -0.530967..."
4,5,blue,"(0.07694420157653206, 0.08043461047646094)",6,"[-0.0073964754, -4.427499, 3.3446236, 4.497534..."


In [11]:
del channel_fr, group_data_map, group_data_map_size

In [12]:
TRANSFORM = "fourier"
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1590/1590 [00:03<00:00, 443.29it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:00<00:00, 37.99it/s] 


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.05687585851000405)",27,"[5.9434114, 43.824703, -6.5293517, -78.00112, ..."
1,2,green,"(0.059498227389561786, 0.06298638865858242)",4,"[-0.10276712, 0.32533062, -1.1434497, -2.99602..."
2,3,green,"(0.06442352540027595, 0.06720566614877052)",4,"[-1.1082836, 2.2659752, 2.353546, -0.46539894,..."
3,4,green,"(0.06987712429686843, 0.07411588266019639)",6,"[-0.36738566, -2.8920639, -2.1771502, -0.17434..."
4,5,green,"(0.07694420157653206, 0.08043461047646094)",6,"[-0.63410753, -3.5660896, 1.2496893, 2.2420774..."


In [13]:
del channel_fr, group_data_map, group_data_map_size

In [14]:
TRANSFORM = "fourier"
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = np.mean(channel_fr[(channel_fr['band'] == group)]['magnitude_endpoints'].iloc[0])
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 1590/1590 [00:03<00:00, 417.94it/s]


[0.05949823 0.06425809 0.06939873 0.07495063 0.08094668 0.08742242
 0.09441621 0.10196951 0.11012707 0.11893723 0.12845221 0.13872839
 0.14982666 0.16181279 0.17475781 0.18873844 0.20383751 0.22014452
 0.23775608 0.25677656 0.27731869 0.29950418 0.32346452 0.34934168
 0.37728901 0.40747213 0.44006991 0.4752755  0.51329754 0.55436134
 0.59871025 0.64660707 0.69833563]


100%|██████████| 33/33 [00:00<00:00, 37.28it/s] 


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.05687585851000405)",27,"[-6.1360106, 35.824085, -12.069928, -92.59161,..."
1,2,gray,"(0.059498227389561786, 0.06298638865858242)",4,"[-0.3495066, 0.6094359, -1.0128437, -3.8297648..."
2,3,gray,"(0.06442352540027595, 0.06720566614877052)",4,"[-1.3334223, 2.6014712, 2.722533, -0.4720191, ..."
3,4,gray,"(0.06987712429686843, 0.07411588266019639)",6,"[-0.17682573, -3.0481725, -1.99498, -0.1464446..."
4,5,gray,"(0.07694420157653206, 0.08043461047646094)",6,"[-0.6237244, -3.9562533, 1.5840019, 2.6535633,..."


In [15]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [34]:
FINAL_DATA_NAME = 'approx1e5-pastis-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"full-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "pastis", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [35]:
TRANSFORM = "wavelet"
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                             
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:03<00:00, 520.15it/s]


,channel,layer,frequency,orientation,data
0,red,1,0.124514,L1,"[-32.279766, 46.440998, -18.512537, -116.88595..."
1,red,2,0.142301,D,"[-20.024885, 2.7007098, 18.920065, 13.724835, ..."
2,red,2,0.142301,H,"[-53.735424, -75.62084, 10.2619915, -9.904851,..."
3,red,2,0.142301,V,"[51.26435, 11.401177, -26.746952, -8.095386, 1..."
4,red,3,0.166018,D,"[33.58143, 3.238668, -7.6894274, 1.0450839, -1..."


In [19]:
del channel_wv, group_data_map, group_data_map_size

In [20]:
TRANSFORM = "wavelet"
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:04<00:00, 397.42it/s]


,channel,layer,frequency,orientation,data
0,green,1,0.124514,L1,"[5.9434114, 43.824703, -6.5293517, -78.00112, ..."
1,green,2,0.142301,D,"[-19.189472, 6.8943515, 17.79731, 12.112439, -..."
2,green,2,0.142301,H,"[-48.65228, -70.861984, 5.356044, -3.6297, 1.2..."
3,green,2,0.142301,V,"[53.264683, 7.8959517, -23.078075, -4.13617, 0..."
4,green,3,0.166018,D,"[33.832684, 3.044488, -9.617887, -3.5956833, -..."


In [21]:
del channel_wv, group_data_map, group_data_map_size

In [22]:
TRANSFORM = "wavelet"
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
    
if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:03<00:00, 398.09it/s]


,channel,layer,frequency,orientation,data
0,blue,1,0.124514,L1,"[0.20727114, -33.177452, -23.717556, -104.1030..."
1,blue,2,0.142301,D,"[-26.155533, 6.1044655, 21.780111, 24.545807, ..."
2,blue,2,0.142301,H,"[-90.20362, -75.99399, 19.310474, -11.829856, ..."
3,blue,2,0.142301,V,"[76.4653, 12.720116, -29.556787, -10.373663, 0..."
4,blue,3,0.166018,D,"[53.602417, 3.668749, -16.509668, -0.57486, -1..."


In [23]:
del channel_wv, group_data_map, group_data_map_size

In [24]:
TRANSFORM = "wavelet"
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    if BATCH_NUM is None:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    else:
        pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))

if BATCH_NUM is None:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
else:
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

8 layers being used


100%|██████████| 1590/1590 [00:05<00:00, 308.49it/s]


,channel,layer,frequency,orientation,data
0,gray,1,0.124514,L1,"[-6.1360106, 35.824085, -12.069928, -92.59161,..."
1,gray,2,0.142301,D,"[-20.23139, 5.5501356, 18.585161, 14.010576, -..."
2,gray,2,0.142301,H,"[-54.903618, -72.86237, 8.412701, -6.4397974, ..."
3,gray,2,0.142301,V,"[55.306328, 9.492829, -24.910969, -6.03024, 0...."
4,gray,3,0.166018,D,"[36.00795, 3.1733897, -9.826171, -1.8638246, -..."


In [25]:
del channel_wv, group_data_map, group_data_map_size